# Model based analysis Reversal learning
#### Korem et al., 2023?

This notebook loads the reversal task from the aging project and uses variations of the Rescorla Wagner (RW) and Pierce Hall (PH) to try and model the effects of aging on learning from reward feedback. 

#### Step 1 model behavior

We will test the following models: </br>
    1. Classical RW </br>
    2. Halves -> different learning parameter for the acquisition and reversal phases </br>
    3. Valence -> different learning parameter for positive and negative updating </br>
    4. decay -> add a decay factor to older data
    4. Valence halves -> different learning parameters for the acquisition and reversal phases and for positive and negative updating
   
#### Step 2 model age

Take the best model and check if we add age parameters are they different than 0


## Step 0

### Load libraries 


In [1]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import scipy, os
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns

import theano
import theano.tensor as tt

import update_q # this is helper functions I used as a seperate file to make the notebook more readable

import pymc as pm
import arviz as az

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Load the data

For the data you need the demographics file, created using a different script and access to the S@Y.

In [2]:
data = pd.read_csv("C:\\Users\\zhb4\\Box Sync\\Neural Computations\\Online Pilot\\Learning_Data_Organized.csv")

In [3]:
data = data[data['sub']>150]
data.head()

,sub,cohort,time,trial,trial_type,mean_stim,actual_stim,val,RT_val,conf,RT_conf,rank
30,239,2,1684378824783,3,reward,50,60,30.0,4.170,9.0,1.449,1.0
31,239,2,1684378835249,4,reward,50,50,60.0,2.143,9.0,1.743,2.0
32,239,2,1684378880564,10,reward,50,70,70.0,2.172,9.0,1.300,3.0
33,239,2,1684378932327,16,reward,50,30,60.0,1.642,9.0,1.853,4.0
34,239,2,1684378990324,23,reward,50,40,60.0,2.569,9.0,1.249,5.0


### Data preparation

extract number of subjects and indexes for modeling 

In [4]:
n_subj   = len(data['sub'].unique())
n_trials = 66 # int(max(data['rank'].values)) # all have 70 trials

n_subj

20

In [5]:
trials, subj = np.meshgrid(list(range(n_trials)), range(n_subj))
# trial = (trials<35)*1 # for modeling alpha for ACQ and reversal phases
trials = tt.as_tensor_variable(trials.T)
# trial = tt.as_tensor_variable(trial.T)
subj   = tt.as_tensor_variable(subj.T)

In [25]:
stim   = np.reshape([data['trial_type']],   (n_subj, n_trials)).T # tilt of the X (0 or 45)
reward = np.reshape([data['actual_stim']], (n_subj, n_trials)).T # was there a reward (0 or 6)
rating = np.reshape([data['val']],    (n_subj, n_trials)).T # response 1-9
rating = [np.nan if 0 else (x+100)/200 for x in rating]

In [26]:
stim   = np.array(stim=='reward',  dtype='int')
reward = np.array((reward+100)/200)

stim = tt.as_tensor_variable(stim)
reward = tt.as_tensor_variable(reward)

In [21]:
reward

TensorConstant{[[0.8  0.8...05 0.1 ]]}

### Tuning parameters

In [9]:
tune = 2000 # 2000
draws = 2000
target_accept = .95 #.99

## Step 1: Modeling behavior

### Model 1: Classical Rescorla Wagner

In [27]:
with pm.Model() as model_RW:
    
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',2) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h  = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta    = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    a_a = pm.TruncatedNormal('a_a', 3, 2, lower=1)
    a_b = pm.TruncatedNormal('a_b', 7, 2, lower=1)

    alpha = pm.Beta('alpha', a_a, a_b, shape=n_subj)
    
    
    Qs  = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn            = update_q.update_Q,
        sequences     = [stim, reward],
        outputs_info  = [Qs, vec],
        non_sequences = [alpha, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW = pm.sample(tune                 = tune, 
                         draws                = draws, 
                         return_inferencedata = True, 
                         target_accept        = target_accept)

AsTensorError: ('Cannot convert alpha to TensorType', <class 'pytensor.tensor.var.TensorVariable'>)

### Model 2:  Rescorla Wagner with separate learning rate for negative and positive updating

Hopkins, A. K., Dolan, R., Button, K. S., & Moutoussis, M. (2021). A reduced self-positive belief underpins greater sensitivity to negative evaluation in socially anxious individuals. Computational psychiatry (Cambridge, Mass.), 5(1), 21.

In [8]:
with pm.Model() as model_RW_valence:
    
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',5) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
     
    beta_h  = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta    = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    ap_a = pm.TruncatedNormal('ap_a', 3, 2, lower=1)
    ap_b = pm.TruncatedNormal('ap_b', 7, 2, lower=1)
    
    an_a = pm.TruncatedNormal('an_a', 3, 2, lower=1)
    an_b = pm.TruncatedNormal('an_b', 7, 2, lower=1)

    alpha_p = pm.Beta('alpha_p', ap_a, ap_b, shape=n_subj)
    alpha_n = pm.Beta('alpha_n', an_a, an_b, shape=n_subj)
    
    
    Qs  = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn            = update_q.update_Q_valence,
        sequences     = [stim, reward],
        outputs_info  = [Qs, vec],
        non_sequences = [alpha_p, alpha_n, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_valence = pm.sample(tune                 = tune, 
                                 draws                = draws, 
                                 return_inferencedata = True, 
                                 target_accept        = target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_n, alpha_p, an_b, an_a, ap_b, ap_a, eps, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 2454 seconds.
The number of effective samples is smaller than 10% for some parameters.


### Model 3:  Rescorla Wagner with separate learning rate for the different phases

Daw, N. D. (2011). Trial-by-trial data analysis using computational models. Decision making, affect, and learning: Attention and performance XXIII, 23(1).

In [9]:
with pm.Model() as model_RW_halves:
    
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',1) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    a_a1 = pm.TruncatedNormal('a_a1', 3, 1, lower=1)
    a_b1 = pm.TruncatedNormal('a_b1', 8, 2, lower=1)
    
    a_a2 = pm.TruncatedNormal('a_a2', 3, 1, lower=1)
    a_b2 = pm.TruncatedNormal('a_b2', 8, 2, lower=1)


    alpha_1 = pm.Beta('alpha_1', a_a1, a_b1, shape=n_subj)
    alpha_2 = pm.Beta('alpha_2', a_a2, a_b2, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    Qs  = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn            = update_q.update_Q_half,
        sequences     = [stim, reward, trial],
        outputs_info  = [Qs, vec],
        non_sequences = [alpha_1, alpha_2,  n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_halves = pm.sample(tune                 = tune, 
                                draws                = draws, 
                                return_inferencedata = True, 
                                target_accept        = target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, alpha_2, alpha_1, a_b2, a_a2, a_b1, a_a1, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 2107 seconds.
The number of effective samples is smaller than 25% for some parameters.


### Model 4: Rescorla Wagner with separate learning rate for negative and positive updating and the different phases

In [10]:
with pm.Model() as model_RW_valence_halves:
    
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',1) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h  = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta    = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    ap_a1 = pm.TruncatedNormal('ap_a1', 3, 1, lower=1)
    ap_b1 = pm.TruncatedNormal('ap_b1', 8, 2, lower=1)
    
    ap_a2 = pm.TruncatedNormal('ap_a2', 3, 1, lower=1)
    ap_b2 = pm.TruncatedNormal('ap_b2', 8, 2, lower=1)
    
    an_a1 = pm.TruncatedNormal('an_a1', 3, 1, lower=1)
    an_b1 = pm.TruncatedNormal('an_b1', 8, 2, lower=1)
    
    an_a2 = pm.TruncatedNormal('an_a2', 3, 1, lower=1)
    an_b2 = pm.TruncatedNormal('an_b2', 8, 2, lower=1)

    alphap_1 = pm.Beta('alphap_1', ap_a1, ap_b1, shape=n_subj)
    alphap_2 = pm.Beta('alphap_2', ap_a2, ap_b2, shape=n_subj)
    alphan_1 = pm.Beta('alphan_1', an_a1, an_b1, shape=n_subj)
    alphan_2 = pm.Beta('alphan_2', an_a2, an_b2, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    Qs  = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn            = update_q.update_Q_valence_half,
        sequences     = [stim, reward, trial],
        outputs_info  = [Qs, vec],
        non_sequences = [alphap_1, alphan_1, alphap_2, alphan_2,  n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_valence_halves = pm.sample(tune                 = tune, 
                                        draws                = draws, 
                                        return_inferencedata = True, 
                                        target_accept        = target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, alphan_2, alphan_1, alphap_2, alphap_1, an_b2, an_a2, an_b1, an_a1, ap_b2, ap_a2, ap_b1, ap_a1, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 2963 seconds.
The number of effective samples is smaller than 10% for some parameters.


### Model 5: Rescorla Wagner with a decay factor

cite

In [11]:
with pm.Model() as model_RW_decay:
    
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',1) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h  = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta    = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    a_a = pm.TruncatedNormal('a_a', 3, 1, lower=1)
    a_b = pm.TruncatedNormal('a_b', 8, 2, lower=1)

    alpha = pm.Beta('alpha', a_a, a_b, shape=n_subj)
    
    decay_a = pm.TruncatedNormal('decay_a', 5, 2, lower=1)
    decay_b = pm.TruncatedNormal('decay_b', 2, 2, lower=1)
    
    decay = pm.Beta('decay', decay_a, decay_b, shape=n_subj)
    
    Qs  = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn            = update_q.update_Q_decay,
        sequences     = [stim, reward],
        outputs_info  = [Qs, vec],
        non_sequences = [alpha, decay, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed = rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_decay = pm.sample(tune                 = tune, 
                               draws                = draws, 
                               return_inferencedata = True, 
                               target_accept        = .99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [decay, decay_b, decay_a, alpha, a_b, a_a, eps, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 4992 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


### Model 6: Pierce Hall model

Homan, P., Levy, I., Feltham, E., Gordon, C., Hu, J., Li, J., ... & Schiller, D. (2019). Neural computations of threat in the aftermath of combat trauma. Nature Neuroscience, 22(3), 470-476.

In [12]:
with pm.Model() as model_PH:
    
    # Hyper priors
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',1) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', .1)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    
    eta_a = pm.TruncatedNormal('eta_a', 3, 1, lower=1)
    eta_b = pm.TruncatedNormal('eta_b', 8, 2, lower=1)
    
    kappa_a = pm.TruncatedNormal('kappa_a', 8, 2, lower=1)
    kappa_b = pm.TruncatedNormal('kappa_b', 3, 1, lower=1)
      
    eta = pm.Beta('eta', eta_a, eta_b, shape=n_subj)
    kappa = pm.Beta('kappa', kappa_a, kappa_b, shape=n_subj)

    
    eps = pm.HalfNormal('eps', 5)
    
    
    Qs = 0.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 0.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    alpha = 0 * tt.ones((n_subj,2), dtype='float64') # vector to save the relevant stimulus's expactation
    assoc = 0 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec, alpha, assoc], updates = theano.scan(
        fn=update_q.update_Q_hall,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, alpha, assoc],
        non_sequences=[eta, kappa, n_subj])
    
    vec_ = vec[trials, subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', mu = vec_, sd = eps, observed=rating) 
    
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_PH = pm.sample(tune=3000, draws=draws, return_inferencedata=True, target_accept= .99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, kappa, eta, kappa_b, kappa_a, eta_b, eta_a, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 3_000 tune and 2_000 draw iterations (12_000 + 8_000 draws total) took 11594 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


### Compare models

In [13]:
models = {'RW': trace_RW, 
          'RW valence': trace_RW_valence,
          'RW halves':trace_RW_halves,
          'RW decay':trace_RW_decay, 
          'RW valence halves':trace_RW_valence_halves,
          'PH': trace_PH}

comp = az.compare(models)
az.compare(models, ic='loo')

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
RW valence halves,0,-520.368251,144.714920,0.000000,5.856486e-01,41.378988,0.000000,False,log
RW halves,1,-532.017050,140.661940,11.648798,4.143514e-01,41.119988,12.512387,False,log
RW valence,2,-569.766451,106.962732,49.398200,1.581362e-11,40.143034,10.210727,False,log
RW,3,-595.539724,95.791900,75.171472,2.450948e-12,39.557369,11.128312,False,log
PH,4,-603.393320,79.596104,83.025069,6.918079e-12,39.363098,12.130089,False,log
RW decay,5,-603.978041,89.387339,83.609790,0.000000e+00,39.446025,11.935965,False,log


## Step 2: Is there an association between age and learning rate?

### Model 7: Classical Rescorla Wagner + Age

In [14]:
with pm.Model() as model_RW_Age:
    
    # connecting function
    mu = pm.Normal('mu', .4, .1)
    sd = pm.HalfNormal('sd',.5) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', .3, .1)
    beta_sd = pm.HalfNormal('beta_sd', .5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    # Age slope
    Age = pm.Normal('Age', 0, .2)
    
    # Model Hyper priors
    a_a = pm.TruncatedNormal('a_a', 3, 2, lower=1)
    a_b = pm.TruncatedNormal('a_b', 7, 2, lower=1)
    eps = pm.HalfNormal('eps', 5)
    
    alpha = pm.Beta('alpha', a_a, a_b, shape=n_subj)
    alphaAge = alpha + Age*ageList
    
    Qs = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_q.update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alphaAge, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_age = pm.sample(tune=tune, draws=draws, return_inferencedata=True, target_accept= target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, eps, a_b, a_a, Age, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 4281 seconds.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


### Model 8:  Rescorla Wagner with separate learning rate for negative and positive updating + Age

In [15]:
with pm.Model() as model_RW_valence_age:
    
    mu = pm.Normal('mu', .4, .1)
    sd = pm.HalfNormal('sd',.5) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', .3, .1)
    beta_sd = pm.HalfNormal('beta_sd', .5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    ap_a = pm.TruncatedNormal('ap_a', 3, 2, lower=1)
    ap_b = pm.TruncatedNormal('ap_b', 7, 2, lower=1)
    
    an_a = pm.TruncatedNormal('an_a', 3, 2, lower=1)
    an_b = pm.TruncatedNormal('an_b', 7, 2, lower=1)

    alpha_p = pm.Beta('alpha_p', ap_a, ap_b, shape=n_subj)
    alpha_n = pm.Beta('alpha_n', an_a, an_b, shape=n_subj)
    
    Agep = pm.Normal('Agep', 0, .2)
    Agen = pm.Normal('Agen', 0, .2)
    
    alphapAge = alpha_p + Agep*ageList
    alphanAge = alpha_n + Agen*ageList
    
    
    Qs = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_q.update_Q_valence,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alphapAge, alphanAge, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_valence_age = pm.sample(tune=tune, draws=draws, return_inferencedata=True, target_accept= target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Agen, Agep, alpha_n, alpha_p, an_b, an_a, ap_b, ap_a, eps, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 3877 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


### Model 9:  Rescorla Wagner with separate learning rate for the different phases + Age


In [16]:
with pm.Model() as model_RW_halves_h:
    
    mu = pm.Normal('mu', .4, .1)
    sd = pm.HalfNormal('sd',.5) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', .3, .1)
    beta_sd = pm.HalfNormal('beta_sd', .5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    a_a1 = pm.TruncatedNormal('a_a1', 3, 1, lower=1)
    a_b1 = pm.TruncatedNormal('a_b1', 8, 2, lower=1)
    
    a_a2 = pm.TruncatedNormal('a_a2', 3, 1, lower=1)
    a_b2 = pm.TruncatedNormal('a_b2', 8, 2, lower=1)


    alpha_1 = pm.Beta('alpha_1', a_a1, a_b1, shape=n_subj)
    alpha_2 = pm.Beta('alpha_2', a_a2, a_b2, shape=n_subj)
    
    Age1 = pm.Normal('Age1', 0, .2)
    Age2 = pm.Normal('Age2', 0, .2)
    
    alpha1Age = alpha_1 + Age1*ageList
    alpha2Age = alpha_2 + Age2*ageList
    
    eps = pm.HalfNormal('eps', 5)
    
    Qs = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_q.update_Q_half,
        sequences=[stim, reward, trial],
        outputs_info=[Qs, vec],
        non_sequences=[alpha1Age, alpha2Age,  n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_halves_age = pm.sample(tune=tune, draws=draws, return_inferencedata=True, target_accept= target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, Age2, Age1, alpha_2, alpha_1, a_b2, a_a2, a_b1, a_a1, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 3113 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


### Model 10: Rescorla Wagner with separate learning rate for negative and positive updating and the different phases + Age

In [7]:
with pm.Model() as model_RW_valence_halves:
    
    mu = pm.Normal('mu', .4, .1)
    sd = pm.HalfNormal('sd',.5) 
    intercept = pm.Normal('intercept', mu, sd, shape=n_subj)
    
    beta_h = pm.Normal('beta_h', .3, .1)
    beta_sd = pm.HalfNormal('beta_sd', .5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    ap_a1 = pm.TruncatedNormal('ap_a1', 3, 1, lower=1)
    ap_b1 = pm.TruncatedNormal('ap_b1', 8, 1, lower=1)
    
    ap_a2 = pm.TruncatedNormal('ap_a2', 3, 1, lower=1)
    ap_b2 = pm.TruncatedNormal('ap_b2', 8, 1, lower=1)
    
    an_a1 = pm.TruncatedNormal('an_a1', 3, 1, lower=1)
    an_b1 = pm.TruncatedNormal('an_b1', 8, 1, lower=1)
    
    an_a2 = pm.TruncatedNormal('an_a2', 3, 1, lower=1)
    an_b2 = pm.TruncatedNormal('an_b2', 8, 1, lower=1)

    alphap_1 = pm.Beta('alphap_1', ap_a1, ap_b1, shape=n_subj)
    alphap_2 = pm.Beta('alphap_2', ap_a2, ap_b2, shape=n_subj)
    alphan_1 = pm.Beta('alphan_1', an_a1, an_b1, shape=n_subj)
    alphan_2 = pm.Beta('alphan_2', an_a2, an_b2, shape=n_subj)
    
    Age1p = pm.Normal('Age1p', 0, .2)
    Age2p = pm.Normal('Age2p', 0, .2)
    Age1n = pm.Normal('Age1n', 0, .2)
    Age2n = pm.Normal('Age2n', 0, .2)
    
    alpha1pAge = alphap_1 + Age1p*ageList
    alpha2pAge = alphap_2 + Age2p*ageList
    alpha1nAge = alphan_1 + Age1n*ageList
    alpha2nAge = alphan_2 + Age2n*ageList
    
    eps = pm.HalfNormal('eps', 5)
    
    Qs = .5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = .5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_q.update_Q_valence_half,
        sequences=[stim, reward, trial],
        outputs_info=[Qs, vec],
        non_sequences=[alpha1pAge, alpha1nAge, alpha2pAge, alpha2nAge,  n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    learn = pm.Normal('learn', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trace_RW_valence_halves_age = pm.sample(tune=tune, draws=draws, return_inferencedata=True, target_accept= target_accept)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, Age2n, Age1n, Age2p, Age1p, alphan_2, alphan_1, alphap_2, alphap_1, an_b2, an_a2, an_b1, an_a1, ap_b2, ap_a2, ap_b1, ap_a1, beta, beta_sd, beta_h, intercept, sd, mu]


Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 5352 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 31 divergences after tuning. Increase `target_accept` or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.


In [19]:
models = {'RW age': trace_RW_age, 
          'RW valence age': trace_RW_valence_age,
          'RW halves age': trace_RW_halves_age,
          'RW valence halves age': trace_RW_valence_halves_age}

comp = az.compare(models)
az.compare(models, ic='loo')

/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/arviz/stats/stats.py:694: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/arviz/stats/stats.py:694: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/home/nachshon/anaconda3/envs/reve

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
RW valence halves age,0,-495.239646,181.385978,0.000000,5.190995e-01,42.426437,0.000000,True,log
RW halves age,1,-500.069867,153.255492,4.830221,4.558511e-01,42.134383,11.678149,False,log
RW valence age,2,-523.059417,149.543599,27.819771,2.504947e-02,41.447206,9.702438,True,log
RW age,3,-586.568625,105.704994,91.328978,2.794875e-12,39.859164,13.987391,True,log


In [9]:
az.summary(trace_RW_valence_halves_age, var_names=['Age1p', 'Age1n', 'Age2p', 'Age2n'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Age1p,-0.281,0.051,-0.377,-0.190,0.002,0.001,1039.0,1797.0,1.00
Age1n,-0.159,0.065,-0.286,-0.061,0.003,0.002,478.0,1076.0,1.01
Age2p,-0.333,0.074,-0.470,-0.194,0.002,0.002,1235.0,2480.0,1.00
Age2n,-0.100,0.036,-0.170,-0.038,0.001,0.001,976.0,1007.0,1.00
